In [1]:
import pickle
import os
import re
import numpy as np
from gensim.models import word2vec
import logging
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '../Core-scripts/')

from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [7]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [4]:
yeast_strict_real = pickle.load(open('../../Results/Yeast/yeast_mentions_strict_real.pkl', 'rb'))
yeast_gen_real = pickle.load(open('../../Results/Yeast/yeast_mentions_gen_real.pkl', 'rb'))
yeast_be_real = pickle.load(open('../../Results/Yeast/yeast_mentions_be_real.pkl', 'rb'))
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [10]:
for seed in random_seeds:
    real_tr_te_name = 'Yeast/train_test/yeast_tr_te_split_' + str(seed)
    train_data, b, c, d = pred.manual_train_test_split(yeast_strict_real, real_tr_te_name, random_state=seed ,test_set_prop=0.1)

In [5]:
yeast_w2v_model_strict = pred.make_w2v_model(yeast_strict_real, 'Yeast/models/yeast_strict')
yeast_w2v_model_gen = pred.make_w2v_model(yeast_gen_real, 'Yeast/models/yeast_gen')
yeast_w2v_model_be = pred.make_w2v_model(yeast_be_real, 'Yeast/models/yeast_be')

2017-05-27 15:31:53,355 : INFO : collecting all words and their counts
2017-05-27 15:31:53,356 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-27 15:31:53,401 : INFO : collected 11431 word types from a corpus of 188505 raw words and 7145 sentences
2017-05-27 15:31:53,402 : INFO : Loading a fresh vocabulary
2017-05-27 15:31:53,419 : INFO : min_count=6 retains 2978 unique words (26% of original 11431, drops 8453)
2017-05-27 15:31:53,420 : INFO : min_count=6 leaves 173585 word corpus (92% of original 188505, drops 14920)
2017-05-27 15:31:53,430 : INFO : deleting the raw counts dictionary of 11431 items
2017-05-27 15:31:53,431 : INFO : sample=0.0001 downsamples 549 most-common words
2017-05-27 15:31:53,432 : INFO : downsampling leaves estimated 74593 word corpus (43.0% of prior 173585)
2017-05-27 15:31:53,432 : INFO : estimated required memory for 2978 words and 600 dimensions: 23526200 bytes
2017-05-27 15:31:53,437 : INFO : constructing a huffman tree fr

Parsing datasets sentences
Training Word2Vec Model


2017-05-27 15:31:53,513 : INFO : built huffman tree with maximum node depth 15
2017-05-27 15:31:53,520 : INFO : resetting layer weights
2017-05-27 15:31:53,569 : INFO : training model with 4 workers on 2978 vocabulary and 600 features, using sg=1 hs=1 sample=0.0001 negative=5 window=7
2017-05-27 15:31:53,569 : INFO : expecting 7145 sentences, matching count from corpus used for vocabulary survey
2017-05-27 15:31:54,868 : INFO : PROGRESS: at 13.77% examples, 39538 words/s, in_qsize 6, out_qsize 1
2017-05-27 15:31:56,040 : INFO : PROGRESS: at 30.70% examples, 46446 words/s, in_qsize 7, out_qsize 0
2017-05-27 15:31:57,088 : INFO : PROGRESS: at 43.38% examples, 46099 words/s, in_qsize 7, out_qsize 0
2017-05-27 15:31:58,095 : INFO : PROGRESS: at 59.33% examples, 48993 words/s, in_qsize 7, out_qsize 0
2017-05-27 15:31:59,177 : INFO : PROGRESS: at 72.02% examples, 47957 words/s, in_qsize 8, out_qsize 0
2017-05-27 15:32:00,231 : INFO : PROGRESS: at 85.75% examples, 48079 words/s, in_qsize 7, o

Parsing datasets sentences


2017-05-27 15:32:01,839 : INFO : collecting all words and their counts
2017-05-27 15:32:01,840 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-27 15:32:01,880 : INFO : PROGRESS: at sentence #10000, processed 241629 words, keeping 11026 word types
2017-05-27 15:32:01,921 : INFO : PROGRESS: at sentence #20000, processed 486297 words, keeping 15003 word types
2017-05-27 15:32:01,963 : INFO : PROGRESS: at sentence #30000, processed 729082 words, keeping 17548 word types
2017-05-27 15:32:02,005 : INFO : PROGRESS: at sentence #40000, processed 971013 words, keeping 19661 word types
2017-05-27 15:32:02,037 : INFO : collected 21147 word types from a corpus of 1155938 raw words and 47662 sentences
2017-05-27 15:32:02,038 : INFO : Loading a fresh vocabulary


...

2017-05-27 15:32:02,064 : INFO : min_count=6 retains 7355 unique words (34% of original 21147, drops 13792)
2017-05-27 15:32:02,064 : INFO : min_count=6 leaves 1130505 word corpus (97% of original 1155938, drops 25433)
2017-05-27 15:32:02,082 : INFO : deleting the raw counts dictionary of 21147 items
2017-05-27 15:32:02,084 : INFO : sample=0.0001 downsamples 521 most-common words
2017-05-27 15:32:02,085 : INFO : downsampling leaves estimated 513926 word corpus (45.5% of prior 1130505)
2017-05-27 15:32:02,085 : INFO : estimated required memory for 7355 words and 600 dimensions: 58104500 bytes
2017-05-27 15:32:02,097 : INFO : constructing a huffman tree from 7355 words
2017-05-27 15:32:02,271 : INFO : built huffman tree with maximum node depth 18
2017-05-27 15:32:02,286 : INFO : resetting layer weights
2017-05-27 15:32:02,399 : INFO : training model with 4 workers on 7355 vocabulary and 600 features, using sg=1 hs=1 sample=0.0001 negative=5 window=7
2017-05-27 15:32:02,399 : INFO : expec

Parsing datasets sentences


2017-05-27 15:33:01,565 : INFO : collecting all words and their counts
2017-05-27 15:33:01,566 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-27 15:33:01,604 : INFO : PROGRESS: at sentence #10000, processed 228892 words, keeping 12369 word types
2017-05-27 15:33:01,651 : INFO : PROGRESS: at sentence #20000, processed 457153 words, keeping 17053 word types
2017-05-27 15:33:01,702 : INFO : PROGRESS: at sentence #30000, processed 685749 words, keeping 20407 word types
2017-05-27 15:33:01,750 : INFO : PROGRESS: at sentence #40000, processed 916505 words, keeping 23089 word types


Training Word2Vec Model


2017-05-27 15:33:01,799 : INFO : PROGRESS: at sentence #50000, processed 1145300 words, keeping 25362 word types
2017-05-27 15:33:01,874 : INFO : PROGRESS: at sentence #60000, processed 1373137 words, keeping 27351 word types
2017-05-27 15:33:01,935 : INFO : PROGRESS: at sentence #70000, processed 1602013 words, keeping 29061 word types
2017-05-27 15:33:01,986 : INFO : PROGRESS: at sentence #80000, processed 1831766 words, keeping 30664 word types
2017-05-27 15:33:02,051 : INFO : PROGRESS: at sentence #90000, processed 2060233 words, keeping 32183 word types
2017-05-27 15:33:02,112 : INFO : PROGRESS: at sentence #100000, processed 2289027 words, keeping 33598 word types
2017-05-27 15:33:02,165 : INFO : PROGRESS: at sentence #110000, processed 2517818 words, keeping 34978 word types
2017-05-27 15:33:02,215 : INFO : PROGRESS: at sentence #120000, processed 2745192 words, keeping 36222 word types
2017-05-27 15:33:02,258 : INFO : PROGRESS: at sentence #130000, processed 2975232 words, keep

In [8]:
for seed in random_seeds:
    data_name = '../../Results/Yeast/train_test/yeast_tr_te_split_' + str(seed)
    train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
    train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
    test_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
    test_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_strict, feature_count=600)

    strict_list_SR_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                 train_labels, test_labels]

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_gen, feature_count=600)

    strict_list_GEN_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                  train_labels, test_labels]

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_be, feature_count=600)

    strict_list_BE_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                 train_labels, test_labels]

    pickle.dump(strict_list_SR_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_GEN_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_BE_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'wb'))

In [6]:
def modelfit(alg, train_vecs, train_labels, w2v_model_type, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param=alg.get_xgb_params()
        xgtrain = xgb.DMatrix(train_vecs, 
                              label=train_labels)
        cvresult = xgb.cv(xgb_param, 
                          xgtrain, 
                          num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds, 
                          metrics='auc', 
                          early_stopping_rounds=50)
        alg.set_params(n_estimators=cvresult.shape[0])
        
    #fit the algorithm on the data
    alg.fit(train_vecs, train_labels, eval_metric='auc')
    
    #Predict training set:
    train_predictions = alg.predict(train_vecs)
    train_predprob = alg.predict_proba(train_vecs)[:,1]
    
    #Print Model report:
    print(w2v_model_type, '\nModel Report')
    print(w2v_model_type, 'Accuracy: %.4g' % metrics.accuracy_score(train_labels, train_predictions))
    print(w2v_model_type, 'AUC Score (Train): %f' % metrics.roc_auc_score(train_labels, train_predprob))
    
#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importance')
#     plt.ylabel('Feature Importance Score')

    error = 1-metrics.accuracy_score(train_labels, train_predictions)
    auc = metrics.roc_auc_score(train_labels, train_predprob)
    
    return error, auc

In [7]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=5, 
                     min_child_weight=1, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

strict_error_list = []
strict_auc_list = []
gen_error_list = []
gen_auc_list = []
be_error_list = []
be_auc_list = []


for seed in random_seeds:  
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    print(seed)
    strict_error, strict_auc = modelfit(xgb1, strict_list[0], strict_list[2], 'STRICT')
    gen_error, gen_auc = modelfit(xgb1, gen_list[0], gen_list[2], 'GEN')
    be_error, be_auc = modelfit(xgb1, be_list[0], be_list[2], 'BE')
    
    strict_error_list.append(strict_error)
    strict_auc_list.append(strict_auc)
    gen_error_list.append(gen_error)
    gen_auc_list.append(gen_auc)
    be_error_list.append(be_error)
    be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.6086
STRICT AUC Score (Train): 0.607743
GEN 
Model Report
GEN Accuracy: 0.5843
GEN AUC Score (Train): 0.581177
BE 
Model Report
BE Accuracy: 0.5965
BE AUC Score (Train): 0.598749
235
STRICT 
Model Report
STRICT Accuracy: 0.6004
STRICT AUC Score (Train): 0.614761
GEN 
Model Report
GEN Accuracy: 0.6044
GEN AUC Score (Train): 0.599407
BE 
Model Report
BE Accuracy: 0.6018
BE AUC Score (Train): 0.592301
905
STRICT 
Model Report
STRICT Accuracy: 0.6131
STRICT AUC Score (Train): 0.627748
GEN 
Model Report
GEN Accuracy: 0.5965
GEN AUC Score (Train): 0.603445
BE 
Model Report
BE Accuracy: 0.5879
BE AUC Score (Train): 0.571791
2895
STRICT 
Model Report
STRICT Accuracy: 0.5841
STRICT AUC Score (Train): 0.587349
GEN 
Model Report
GEN Accuracy: 0.5781
GEN AUC Score (Train): 0.560142
BE 
Model Report
BE Accuracy: 0.5917
BE AUC Score (Train): 0.584521
3462
STRICT 
Model Report
STRICT Accuracy: 0.5953
STRICT AUC Score (Train): 0.613894
GEN 
Model Report
GEN 

In [9]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=5, 
                     min_child_weight=1, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

strict_error_list = []
strict_auc_list = []
gen_error_list = []
gen_auc_list = []
be_error_list = []
be_auc_list = []


for seed in random_seeds:  
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    print(seed)
    strict_error, strict_auc = modelfit(xgb1, strict_list[0], strict_list[2], 'STRICT')
    gen_error, gen_auc = modelfit(xgb1, gen_list[0], gen_list[2], 'GEN')
    be_error, be_auc = modelfit(xgb1, be_list[0], be_list[2], 'BE')
    
    strict_error_list.append(strict_error)
    strict_auc_list.append(strict_auc)
    gen_error_list.append(gen_error)
    gen_auc_list.append(gen_auc)
    be_error_list.append(be_error)
    be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.637
STRICT AUC Score (Train): 0.737125
GEN 
Model Report
GEN Accuracy: 0.6552
GEN AUC Score (Train): 0.748667
BE 
Model Report
BE Accuracy: 0.6712
BE AUC Score (Train): 0.785685
235
STRICT 
Model Report
STRICT Accuracy: 0.6624
STRICT AUC Score (Train): 0.739747
GEN 
Model Report
GEN Accuracy: 0.6504
GEN AUC Score (Train): 0.738340
BE 
Model Report
BE Accuracy: 0.6855
BE AUC Score (Train): 0.784790
905
STRICT 
Model Report
STRICT Accuracy: 0.6512
STRICT AUC Score (Train): 0.746496
GEN 
Model Report
GEN Accuracy: 0.6653
GEN AUC Score (Train): 0.765681
BE 
Model Report
BE Accuracy: 0.6767
BE AUC Score (Train): 0.790130
2895
STRICT 
Model Report
STRICT Accuracy: 0.6491
STRICT AUC Score (Train): 0.744200
GEN 
Model Report
GEN Accuracy: 0.6756
GEN AUC Score (Train): 0.776996
BE 
Model Report
BE Accuracy: 0.687
BE AUC Score (Train): 0.789860
3462
STRICT 
Model Report
STRICT Accuracy: 0.6515
STRICT AUC Score (Train): 0.742703
GEN 
Model Report
GEN Ac

In [7]:
print('Strict Results Error: ', np.mean(strict_error_list), ', Auc: ', np.mean(strict_auc_list))
print('Gen Results Error: ', np.mean(gen_error_list), ', Auc: ', np.mean(gen_auc_list))
print('Be Results Error: ', np.mean(be_error_list), ', Auc: ', np.mean(be_auc_list))

Strict Results Error:  0.330430479588 , Auc:  0.768074209551
Gen Results Error:  0.364579946523 , Auc:  0.707197019221
Be Results Error:  0.344158176436 , Auc:  0.74322406605


In [8]:
param_test1 = {
    'max_depth':[3,4,5,6,7,8,9,10],
    'min_child_weight':[1,2,3,4,5,6]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=140, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8, 
                                                  objective='binary:logistic', 
                                                  nthread=4, 
                                                  scale_pos_weight=1, 
                                                  seed=24), 
                        param_grid=param_test1, 
                        scoring='roc_auc',  
                        iid=False, 
                        cv=5)

In [10]:
strict_param_test1 = []
for seed in [144]:
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(strict_list[0], strict_list[2])
    strict_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [11]:
gen_param_test1 = []
for seed in [144]:
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(gen_list[0], gen_list[2])
    gen_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [12]:
be_param_test1 = []
for seed in [144]:
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(be_list[0], be_list[2])
    be_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [13]:
print(strict_param_test1, gen_param_test1, be_param_test1)

[(144, ({'max_depth': 3, 'min_child_weight': 1}, 0.5184627231701338))] [(144, ({'max_depth': 3, 'min_child_weight': 5}, 0.5033950980591407))] [(144, ({'max_depth': 6, 'min_child_weight': 6}, 0.4984421837963776))]


In [14]:
xgb2 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=3, 
                     min_child_weight=3, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

strict_error_list = []
strict_auc_list = []
gen_error_list = []
gen_auc_list = []
be_error_list = []
be_auc_list = []


for seed in random_seeds:  
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
    print(seed)
    strict_error, strict_auc = modelfit(xgb2, strict_list[0], strict_list[2], 'STRICT')
    gen_error, gen_auc = modelfit(xgb2, gen_list[0], gen_list[2], 'GEN')
    be_error, be_auc = modelfit(xgb2, be_list[0], be_list[2], 'BE')
    
    strict_error_list.append(strict_error)
    strict_auc_list.append(strict_auc)
    gen_error_list.append(gen_error)
    gen_auc_list.append(gen_auc)
    be_error_list.append(be_error)
    be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.712
STRICT AUC Score (Train): 0.869563
GEN 
Model Report
GEN Accuracy: 0.6677
GEN AUC Score (Train): 0.787012
BE 
Model Report
BE Accuracy: 0.5891
BE AUC Score (Train): 0.640011
235
STRICT 
Model Report
STRICT Accuracy: 0.581
STRICT AUC Score (Train): 0.660918
GEN 
Model Report
GEN Accuracy: 0.586
GEN AUC Score (Train): 0.627057
BE 
Model Report
BE Accuracy: 0.5887
BE AUC Score (Train): 0.651093
905
STRICT 
Model Report
STRICT Accuracy: 0.5811
STRICT AUC Score (Train): 0.656544
GEN 
Model Report
GEN Accuracy: 0.5906
GEN AUC Score (Train): 0.629023
BE 
Model Report
BE Accuracy: 0.5933
BE AUC Score (Train): 0.649648
2895
STRICT 
Model Report
STRICT Accuracy: 0.5872
STRICT AUC Score (Train): 0.674281
GEN 
Model Report
GEN Accuracy: 0.5776
GEN AUC Score (Train): 0.640244
BE 
Model Report
BE Accuracy: 0.5947
BE AUC Score (Train): 0.655433
3462
STRICT 
Model Report
STRICT Accuracy: 0.5994
STRICT AUC Score (Train): 0.673769
GEN 
Model Report
GEN Acc